# Etude à l'échelle de la consommation par adresse

Dans ce notebook, on cherche à récupérer les données météo de plusieurs communes, afin de rendre pertinente une régression de la consommation d'énergie en fonction de l'indice de performance du bâtiment et de la météo.

Cette analyse s'inscrit dans une dimension bien plus locale que l'analyse précédente, qui portait sur une échelle départementale et non communale.

## Mise en route

In [4]:
!pip install pandas
!pip install geopandas
!pip install geoviews
!pip install lxml
!pip install urllib
!pip install matplotlib
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a
!pip install mapclassify
!pip install folium

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
  Cloning https://github.com/inseefrlab/cartiflette (to revision 80b8a5a28371feb6df31d55bcc2617948a5f9b1a) to /tmp/pip-req-build-r0avumpt
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartiflette /tmp/pip-req-build-r0avumpt
  Running command git rev-parse -q --verify 'sha^80b8a5a28371feb6df31d55bcc2617948a5f9b1a'
  Running command git fetch -q https://github.com/inseefrlab/cartiflette 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Running command git checkout -q 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Resolved https://github.com/inseefrlab/cartiflette to commit 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
!pip install -q lxml

In [6]:
from declarations import *

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 63.9MiB/s]


In [7]:
import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

## Récupération des données de consommation individuelles

In [8]:
import declarations as dec

df_cons_2018=dec.get_data_consumption(dec.consumption_data_url_2018, "2018")
df_cons_2019=dec.get_data_consumption(dec.consumption_data_url_2019, "2019")
df_cons_2020=dec.get_data_consumption(dec.consumption_data_url_2020, "2020")
df_cons_2021=dec.get_data_consumption(dec.consumption_data_url_2021, "2021")

/home/onyxia/work/Projet_Python/declarations.py:98: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")
/home/onyxia/work/Projet_Python/declarations.py:98: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


In [12]:
df_cons_2018


,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses,Code EPCI,Code Département,Code Région
0,2018,940010000,Ablon-sur-Seine (commune non irisée),18,NaN,ALLEE,DES ORMES,94001,ABLON-SUR-SEINE,RESIDENTIEL,10,25.816,2.582,3.880,18 ALLEE DES ORMES,400010,200054781.0,94.0,11.0
1,2018,940010000,Ablon-sur-Seine (commune non irisée),40,NaN,RUE,DU BAC,94001,ABLON-SUR-SEINE,RESIDENTIEL,18,81.973,4.554,3.880,40 RUE DU BAC,400017,200054781.0,94.0,11.0
2,2018,940010000,Ablon-sur-Seine (commune non irisée),19,NaN,RUE,DU BAC,94001,ABLON-SUR-SEINE,RESIDENTIEL,40,151.116,3.778,3.880,19 RUE DU BAC,400021,200054781.0,94.0,11.0
3,2018,940010000,Ablon-sur-Seine (commune non irisée),5,NaN,RUE,DU CNL PIERRE BROSSOLETTE,94001,ABLON-SUR-SEINE,RESIDENTIEL,13,71.831,5.525,3.880,5 RUE DU CNL PIERRE BROSSOLETTE,400027,200054781.0,94.0,11.0
4,2018,940010000,Ablon-sur-Seine (commune non irisée),16,NaN,RESIDENCE,DU VAL D ABLON,94001,ABLON-SUR-SEINE,RESIDENTIEL,10,29.950,2.995,3.880,16 RESIDENCE DU VAL D ABLON,400031,200054781.0,94.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391094,2018,743150000,Yvoire (commune non irisée),255,NaN,CHEMIN,DES MOTTES,74315,YVOIRE,RESIDENTIEL,10,77.067,7.707,7.256,255 CHEMIN DES MOTTES,790833,200067551.0,74.0,84.0
391095,2018,033210101,Centre Ville,3,NaN,RUE,DE BELLECROIX,3321,YZEURE,RESIDENTIEL,33,149.557,4.532,4.404,3 RUE DE BELLECROIX,790851,200071140.0,3.0,84.0
391096,2018,683860000,Zimmersheim (commune non irisée),23,NaN,RUE,LOUIS PASTEUR,68386,ZIMMERSHEIM,RESIDENTIEL,18,37.803,2.100,7.276,23 RUE LOUIS PASTEUR,790879,200066009.0,68.0,44.0
391097,2018,577640000,Zoufftgen (commune non irisée),39,NaN,RUE,DE LA GARE,57764,ZOUFFTGEN,RESIDENTIEL,17,22.323,1.313,5.711,39 RUE DE LA GARE,790881,245700695.0,57.0,44.0


In [9]:
communes = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "COMMUNE",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

communes["area"] = dep.to_crs(2154).area

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 70.4MiB [00:01, 46.5MiB/s]


In [10]:
communes

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,territoire,geometry,area
0,COMMUNE_0000000009754033,NaN,Connangles,CONNANGLES,43076,Commune simple,137,11,1,43,84,200073419,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((3.61421 45.27206, 3.61377 45.27255, ...",5.774291e+09
1,COMMUNE_0000000009760784,NaN,Vidouze,VIDOUZE,65462,Commune simple,243,13,3,65,76,200072106,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-0.02485 43.43002, -0.02500 43.43012...",7.418552e+09
2,COMMUNE_0000000009742077,NaN,Fouesnant,FOUESNANT,29058,Commune simple,9864,11,4,29,53,242900660,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.97912 47.70392, -3.97897 47...",4.033539e+09
3,COMMUNE_0000000009735245,NaN,Plougrescant,PLOUGRESCANT,22218,Commune simple,1166,27,3,22,53,200065928,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.20273 48.84902, -3.20259 48...",4.720673e+09
4,COMMUNE_0000000009752504,NaN,Montcarra,MONTCARRA,38250,Commune simple,569,24,2,38,84,200068542,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((5.43220 45.61477, 5.43220 45.61438, ...",7.365673e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34821,COMMUNE_0000000009742349,NaN,Brie,BRIE,35041,Commune simple,1006,12,1,35,53,243500634,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.58072 47.94312, -1.58057 47.94420...",NaN
34822,COMMUNE_0000000009742038,NaN,Orgères,ORGERES,35208,Commune simple,5152,12,3,35,53,243500139,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.64908 47.96488, -1.65011 47.96595...",NaN
34823,COMMUNE_0000000009734653,NaN,Castillon-en-Auge,CASTILLON-EN-AUGE,14141,Commune simple,163,19,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.09085 49.04872, 0.09129 49.04872, ...",NaN
34824,COMMUNE_0000000009734657,NaN,Saint-Pierre-en-Auge,SAINT-PIERRE-EN-AUGE,14654,Commune simple,7329,18,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.10231 49.02482, 0.10246 49.02443, ...",NaN


In [15]:
communes['INSEE_COM'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 34826 entries, 0 to 34825
Series name: INSEE_COM
Non-Null Count  Dtype 
--------------  ----- 
34826 non-null  object
dtypes: object(1)
memory usage: 272.2+ KB


In [16]:
communes.nunique()

id            34826
ID                0
NOM           32596
NOM_M         32537
INSEE_COM     34826
STATUT            5
POPULATION     5786
INSEE_CAN        56
INSEE_ARR         9
INSEE_DEP        96
INSEE_REG        13
SIREN_EPCI     1244
source            1
territoire        1
geometry      34826
area             96
dtype: int64

In [17]:
df_cons_2018.nunique()

Année                                                                 1
Code IRIS                                                         18041
Nom IRIS                                                          15180
Numéro de voie                                                     2150
Indice de répétition                                                 18
Type de voie                                                         97
Libellé de voie                                                   47117
Code INSEE de la commune                                           6161
Nom de la commune                                                  6085
Segment de client                                                     1
Nombre de logements                                                 410
Consommation annuelle totale de l'adresse (MWh)                  130608
Consommation annuelle moyenne par logement de l'adresse (MWh)     10840
Consommation annuelle moyenne de la commune (MWh)               

In [14]:
data_2018=communes.merge(df_cons_2018, left_on='INSEE_COM', right_on='Code INSEE de la commune')#[['NOM','geometry', 'area', "Temps d'ensoleillement (jours/an)"]]
data_2018

ValueError: You are trying to merge on object and int64 columns for key 'INSEE_COM'. If you wish to proceed you should use pd.concat

Une fois les données récupérées on peut faire quelques statistiques descriptives (carte de l'intensité des points de consommation etc).

## Scrapping des données météo communales

Comme il s'agit de notre seule source de données météo à l'échelle des communes, ce sont ces données qui nous permettront de définir les villes de France auxquelles nous allons nous intéresser.

In [19]:
""" Création du dictionnaire Racine """

data_all = {
    i : {j : {} for j in range(1,13)} for i in range(2011,2019)
}

In [20]:
""" Scrapping du site infoclimat.fr et organisation des données sous forme de dictionnaire """
# Il faut optimiser la création en ne créant qu'une seule fois le Df lié aux villes

for i in tqdm(range(len(data_all))):
    for y in data_all :
        for m in data_all[y] :
            url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
            raw_text = request.urlopen(url_test).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            tableau = page.find('table', {'id' : 'tableau-releves'})
            rows = tableau.find_all('tr')
    
            df = {
        'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
        'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
        'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
        'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
        'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
        'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
        'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
        "Heure d'ensoleillement du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
        'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                }
    
            data_all[y][m] = df

  0%|          | 0/8 [00:00<?, ?it/s]

In [41]:
meteo_2018={
    #'Villes' : data_all[2018][1]['Villes']
    'Moyenne annuelle de la température minimale extrême du mois' : sum([df_all[2018][i]['Température minimale extrême du mois'] for i in range (1,13)])/12,
    "Température moyenne de l'année" : sum([df_all[2018][i]['Température moyenne du mois'] for i in range (1,13)])/12,
    'Moyenne annuelle de la moyenne des températures maximales du mois' : sum([df_all[2018][i]['Moyenne des températures maximales du mois'] for i in range (1,13)])/12,
    'Moyenne annuelle de la température maximale extrême du mois' : sum([df_all[2018][i]['Température maximale extrême du mois'] for i in range (1,13)])/12,
    "Cumul des précipitations de l'année" : sum([df_all[2018][i]['Cumul de précipitation du mois'] for i in range (1,13)]),
    "Heures d'ensoleillement de l'année" : sum([df_all[2018][i]["Heure d'ensoleillement du mois"] for i in range (1,13)]),
    'Moyenne annuelle des rafales maximales du mois' : sum([df_all[2018][i]['Rafale maximale du mois'] for i in range (1,13)])/12 
            }
meteo_2018

TypeError: unsupported operand type(s) for +: 'int' and 'list'

## Création d'une liste de ville

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

In [22]:
df['Villes']

['Abbeville (80)',
 'Agde (34)',
 'Agde - Le Grau (34)',
 'Agen - La Garenne (47)',
 "Aigrefeuille-d'Aunis (17)",
 "Aigrefeuille-d'Aunis - Château-d'eau (17)",
 'AILLEVILLERS (70)',
 'Aix-en-Provence (13)',
 "Aix-les-Bains (Capitainerie d'Aix Grand Lac) (73)",
 'Aizenay (85)',
 "Ajaccio - Campo dell'Oro (2A)",
 'Albi-Le Séquestre (81)',
 'Alençon - Valframbert (61)',
 'Alenya - Mas Blanc (66)',
 'Almenêches (61)',
 'Ambérieu (01)',
 'Ambérieu-en-Bugey (01)',
 'Ambès (33)',
 'Amiens - Dury (80)',
 'Amuré (79)',
 'Ancey (21)',
 'Angers - Marcé (49)',
 'Angers-Beaucouzé (49)',
 'Anglars-Saint-Félix (12)',
 'Aniane (34)',
 'Annecy-Meythet (74)',
 'ANNECY-MEYTHET (FR)',
 'Annonay (07)',
 'Annot (04)',
 'Aouste-sur-Sye (Saint-Pierre) (26)',
 'Arc-sous-Cicon (25)',
 'Arces (17)',
 'Arcizans-Avant (65)',
 'Argenteuil (95)',
 'Armentières (59)',
 'Arnas (69)',
 'Arthun (42)',
 'Artonne (63)',
 'Arvert (1) (17)',
 'Arvert (2) (17)',
 'Asnières-sur-Oise (95)',
 'Atuona (987)',
 'AUBENAS SA (7)',


On voit que les noms de stations ne sont pas très propres : ils incluent les numéros de département, voire incluent des quartiers ou regroupent plusieurs villes. On cherche donc à nettoyer ces noms, pour qu'ils puissent être comparés aux noms de commune donnée par l'API utile pour les DPE. 

On ne touche pour cela qu'à la liste "Villes" du data frame. Nous remplacerons ensuite la base initiale par la base nettoyée, et supprimons les lignes qui ne nous intéressent pas dans le data frame après nettoyage. 

L'API des DPE qui nous intéresse n'identifie une commune que par son code INSEE. Il faut donc que, dans la base de données de chaque station, je réussisse à merger la commune de la station avec une base de données qui comprend les codes communes INSEE. 

In [23]:
#Création d'un Data Frame à partir de la colonne "Villes" du dictionnaire précédent
stations=pd.DataFrame(df['Villes'],columns=['Ville'])
stations

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
873,[MAE] Lycée Roz Glas - QUIMPERLE (29)
874,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17)
875,[MAE] Lycée Victor Louis - TALENCE (33)
876,[MAE] Lycée Victor Louis - TALENCE (33)


In [24]:
#On ne garde que les stations dans les villes et pas celles de stations de "Météo à l'école" (MAE)
liste_communes=stations[~stations['Ville'].str[0].isin(['['])]
liste_communes

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
781,Xonrupt-Longemer (88)
782,Xonrupt-Longemer - Le Poli (88)
783,Xonrupt-Longemer - Refuge du Sotré (88)
784,Yssingeaux (43)


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [35]:
# Méthode très naïve
def nettoyage(liste):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Argument : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        mot=mot.upper()
        List.append([mot,ville[i+2:-1]])
    
    return List
List=nettoyage(liste_communes['Ville'])
List

[['ABBEVILLE', '80'],
 ['AGDE', '34'],
 ['AGDE - LE GRAU', '34'],
 ['AGEN - LA GARENNE', '47'],
 ["AIGREFEUILLE-D'AUNIS", '17'],
 ["AIGREFEUILLE-D'AUNIS - CHÂTEAU-D'EAU", '17'],
 ['AILLEVILLERS', '70'],
 ['AIX-EN-PROVENCE', '13'],
 ['AIX-LES-BAINS', "Capitainerie d'Aix Grand Lac) (73"],
 ['AIZENAY', '85'],
 ["AJACCIO - CAMPO DELL'ORO", '2A'],
 ['ALBI-LE SÉQUESTRE', '81'],
 ['ALENÇON - VALFRAMBERT', '61'],
 ['ALENYA - MAS BLANC', '66'],
 ['ALMENÊCHES', '61'],
 ['AMBÉRIEU', '01'],
 ['AMBÉRIEU-EN-BUGEY', '01'],
 ['AMBÈS', '33'],
 ['AMIENS - DURY', '80'],
 ['AMURÉ', '79'],
 ['ANCEY', '21'],
 ['ANGERS - MARCÉ', '49'],
 ['ANGERS-BEAUCOUZÉ', '49'],
 ['ANGLARS-SAINT-FÉLIX', '12'],
 ['ANIANE', '34'],
 ['ANNECY-MEYTHET', '74'],
 ['ANNECY-MEYTHET', 'FR'],
 ['ANNONAY', '07'],
 ['ANNOT', '04'],
 ['AOUSTE-SUR-SYE', 'Saint-Pierre) (26'],
 ['ARC-SOUS-CICON', '25'],
 ['ARCES', '17'],
 ['ARCIZANS-AVANT', '65'],
 ['ARGENTEUIL', '95'],
 ['ARMENTIÈRES', '59'],
 ['ARNAS', '69'],
 ['ARTHUN', '42'],
 ['ARTONN

In [33]:
m="allo".upper()
m

'ALLO'

In [27]:
""" Conversion en DataFrame pour exporter """

df_all = pd.DataFrame(data_all)
df_all

,2011,2012,2013,2014,2015,2016,2017,2018
1,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
2,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
3,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
4,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
5,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
6,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
7,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
8,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
9,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
10,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (

Je ne sais pas bien encore comment utiliser ce qui suit.

In [13]:
""" Exportation au format json """

df_all.to_json('data_base.json')

In [14]:
""" Normalisation des variables suivant la nomenclature de la source """

df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

In [15]:
class Meteo :
    def __init__(self):
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(2011,2019)
            }

    def scrap(self):
        for y in self.data_all :
            for m in self.data_all[y] :
                url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_test).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
            'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
            'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def rename(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

## Récupération des données de Diagnostics de Performance Energétique (DPE)

On commence par programmer une fonction qui permet de récupérer des données à partir d'une APL.

In [30]:
def get_dpe_from_url(api_url):
    '''Argument:
        api_url (string) : url de l'API voulue
    Sortie:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

In [ ]:
def get_dpe_dataset(api_root, communes):
    '''
    Args : 
    Returns : 
    '''
    